# Linear programming - A Two Stage Production Planning Problem
Follows the [Two Stage Production Planning Problem](https://coin-or.github.io/pulp/CaseStudies/a_two_stage_production_planning_problem.html) example from the Pulp documentation.

### Problem Description
In a production planning problem, the decision maker must decide how to purchase material, labor, and other resources in order to produce end products to maximize profit.

In this case study, a company named GTC produces wrenches and pliers, subject to the availability of steel, machine capabilities (molding and assembly), labor, and market demand.  
GTC would like to determine how much steel to purchase. Complicating the problem is that the available assembly capacity and the product contribution to earnings are unknown presently, but will be known at the beginning of the next period.

So, in this period, GTC must:
- determine how much steel to purchase.

At the beginning of the next period, after GTC finds out how much assembly capacity is available and the revenue per unit of wrenches and pliers, GTC will determine
- How many wrenches and pliers to produce.

The uncertainty is expressed as one of four possible scenarios, each with equal probability.

Next, we will read in the data. Here, we read in the data as vectors. In actual use, this may be read from databases.  
First, the data elements that do not change with scenarios. These each have two values, one corresponding to wrenches, the other pliers.

In [1]:
import pulp

products = ["wrenches", "pliers"]
price = [130, 100]
steel = [1.5, 1]
molding = [1, 1]       # molding time
assembly = [0.3, 0.5]  # assembly time
capsteel = 27          # capacity of steel
capmolding = 21        # capacity of molding
LB = [0, 0]            # lower bound of production
capacity_ub = [15, 16] # upper bound of production
steelprice = 58

The next set of parameters are those that correspond to the four scenarios.

In [2]:
scenarios = [0, 1, 2, 3]              # scenario numbers
pscenario = [0.25, 0.25, 0.25, 0.25]  # probability of scenarios
wrenchearnings = [160, 160, 90, 90]   # earnings of wrenches
plierearnings = [100, 100, 100, 100]  # earnings of pliers
capassembly = [8, 10, 8, 10]          # capacity of assembly

Next, we will create lists that represent the combination of products and scenarios. These will later be used to create dictionaries for the parameters.

In [3]:
production = [(j, i) for j in scenarios for i in products]                 # production of each product in each scenario
pricescenario = [[wrenchearnings[j], plierearnings[j]] for j in scenarios] # price of each product in each scenario
priceitems = [item for sublist in pricescenario for item in sublist]       # price of each product in each scenario

print("production: ", production)
print("pricescenario: ", pricescenario)
print("priceitems: ", priceitems)

production:  [(0, 'wrenches'), (0, 'pliers'), (1, 'wrenches'), (1, 'pliers'), (2, 'wrenches'), (2, 'pliers'), (3, 'wrenches'), (3, 'pliers')]
pricescenario:  [[160, 100], [160, 100], [90, 100], [90, 100]]
priceitems:  [160, 100, 160, 100, 90, 100, 90, 100]


Next, we use dict(zip(…)) to convert these lists to dictionaries. This is done so that we can refer to parameters by meaningful names.

In [4]:
price_dict = dict(zip(production, priceitems))       # price dictionary: price of each product in each scenario
capacity_dict = dict(zip(products, capacity_ub * 4)) # upper capacity dictionary: capacity of each product
steel_dict = dict(zip(products, steel))              # steel dictionary: steel consumption of each product 
molding_dict = dict(zip(products, molding))          # molding dictionary: molding time of each product
assembly_dict = dict(zip(products, assembly))        # assembly dictionary: assembly time of each product

print("price_dict: ", price_dict)
print("capacity_dict: ", capacity_dict)
print("steel_dict: ", steel_dict)
print("molding_dict: ", molding_dict)
print("assembly_dict: ", assembly_dict)

price_dict:  {(0, 'wrenches'): 160, (0, 'pliers'): 100, (1, 'wrenches'): 160, (1, 'pliers'): 100, (2, 'wrenches'): 90, (2, 'pliers'): 100, (3, 'wrenches'): 90, (3, 'pliers'): 100}
capacity_dict:  {'wrenches': 15, 'pliers': 16}
steel_dict:  {'wrenches': 1.5, 'pliers': 1}
molding_dict:  {'wrenches': 1, 'pliers': 1}
assembly_dict:  {'wrenches': 0.3, 'pliers': 0.5}


To define our decision variables, we use the function pulp.LpVariable.dicts(), which creates dictionaries with associated indexing values.

In [5]:
production_vars = pulp.LpVariable.dicts("production", (scenarios, products), lowBound=0, cat="Continuous")  # production variables: production of each product in each scenario
steelpurchase = pulp.LpVariable("steelpurchase", lowBound=0, cat="Continuous")                              # steel purchase variable: steel purchase

print("production_vars: ", production_vars)
print("steelpurchase: ", steelpurchase)

for item in production_vars.items():
    print(item)

production_vars:  {0: {'wrenches': production_0_wrenches, 'pliers': production_0_pliers}, 1: {'wrenches': production_1_wrenches, 'pliers': production_1_pliers}, 2: {'wrenches': production_2_wrenches, 'pliers': production_2_pliers}, 3: {'wrenches': production_3_wrenches, 'pliers': production_3_pliers}}
steelpurchase:  steelpurchase
(0, {'wrenches': production_0_wrenches, 'pliers': production_0_pliers})
(1, {'wrenches': production_1_wrenches, 'pliers': production_1_pliers})
(2, {'wrenches': production_2_wrenches, 'pliers': production_2_pliers})
(3, {'wrenches': production_3_wrenches, 'pliers': production_3_pliers})


We create the LpProblem and then make the objective function. Note that this is a maximization problem, as the goal is to maximize net revenue.

In [6]:
model= pulp.LpProblem("The Gemstone Tool Problem", pulp.LpMaximize) # gemstone problem: maximize the earnings


c:\Users\torha\.conda\envs\LP_env\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


The objective function is specified using the pulp.lpSum() function. Note that it is added to the problem using +=.

In [7]:
model+= (pulp.lpSum([pscenario[j] * (price_dict[(j, i)] * production_vars[j][i]) for (j, i) in production] - steelpurchase * steelprice), "Total cost") # objective function: maximize the earnings


We then add in constraints. Constraints here in sets based on scenarios and products and are specified using the for i in list: notation. Within each constraint, summations are expressed using list comprehensions. Note that constraints are differentiated from the objective function as each constraint ends in a logical comparison (usually <= or >=, but can be ==) while Finally, here, the file gives each constraint a name which includes the specific scenario or product the constraint applies to.

In [8]:
# Constraints

# Alternative 1: Written out over several steps

# Loop through each scenario
for j in scenarios:
    
    # Initialize the steel, molding, and assembly-time used for this scenario to 0
    steel_used_in_scenario_j = 0
    molding_used_in_scenario_j = 0
    assembly_used_in_scenario_j = 0

    # Loop through each product to calculate the steel, molding, and assembly-time used
    for i in products:
        steel_used_for_product_i = steel_dict[i] * production_vars[j][i]
        steel_used_in_scenario_j += steel_used_for_product_i

        molding_used_for_product_i = molding_dict[i] * production_vars[j][i]
        molding_used_in_scenario_j += molding_used_for_product_i

        assembly_used_for_product_i = assembly_dict[i] * production_vars[j][i]
        assembly_used_in_scenario_j += assembly_used_for_product_i

    print(f"Total steel used in scenario {j}: {steel_used_in_scenario_j}")
    print(f"Total molding used in scenario {j}: {molding_used_in_scenario_j}")
    print(f"Total assembly used in scenario {j}: {assembly_used_in_scenario_j}")

    # Add the constraint that steel, molding, and assembly-time used should not exceed its limits
    model += (steel_used_in_scenario_j - steelpurchase <= 0), ("Steel capacity" + str(j))
    model += (molding_used_in_scenario_j <= capmolding), ("molding capacity" + str(j))
    model += (assembly_used_in_scenario_j <= capassembly[j]), ("assembly capacity" + str(j))

    # Loop through each product for this scenario
    for i in products:
        # Add the constraint that production should not exceed capacity
        model += (production_vars[j][i] <= capacity_dict[i]), ("capacity " + str(i) + str(j))


# # Alternative 2: More compact notation
# for j in scenarios:
#     model += pulp.lpSum(
#         [steel_dict[i] * production_vars[j][i] for i in products]
#     ) - steelpurchase <= 0, ("Steel capacity" + str(j))
#     model += pulp.lpSum(
#         [molding_dict[i] * production_vars[j][i] for i in products]
#     ) <= capmolding, ("molding capacity" + str(j))
#     model += pulp.lpSum(
#         [assembly_dict[i] * production_vars[j][i] for i in products]
#     ) <= capassembly[j], ("assembly capacity" + str(j))
#     for i in products:
#         model += production_vars[j][i] <= capacity_dict[i], (
#             "capacity " + str(i) + str(j)
#         )

model

The_Gemstone_Tool_Problem:
MAXIMIZE
25.0*production_0_pliers + 40.0*production_0_wrenches + 25.0*production_1_pliers + 40.0*production_1_wrenches + 25.0*production_2_pliers + 22.5*production_2_wrenches + 25.0*production_3_pliers + 22.5*production_3_wrenches + -58*steelpurchase + 0.0
SUBJECT TO
Steel_capacity0: production_0_pliers + 1.5 production_0_wrenches
 - steelpurchase <= 0

molding_capacity0: production_0_pliers + production_0_wrenches <= 21

assembly_capacity0: 0.5 production_0_pliers + 0.3 production_0_wrenches <= 8

capacity_wrenches0: production_0_wrenches <= 15

capacity_pliers0: production_0_pliers <= 16

Steel_capacity1: production_1_pliers + 1.5 production_1_wrenches
 - steelpurchase <= 0

molding_capacity1: production_1_pliers + production_1_wrenches <= 21

assembly_capacity1: 0.5 production_1_pliers + 0.3 production_1_wrenches <= 10

capacity_wrenches1: production_1_wrenches <= 15

capacity_pliers1: production_1_pliers <= 16

Steel_capacity2: production_2_pliers + 1.5 p

In [9]:
# The problem data is written to an .lp file
model.writeLP("model.lp")

# The problem is solved using PuLP's choice of Solver
model.solve()
# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[model.status])

# OUTPUT

# Each of the variables is printed with it's resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)
production = [v.varValue for v in model.variables()]

# The optimised objective function value is printed to the console
print("Total price = ", pulp.value(model.objective))

Status: Optimal
production_0_pliers = 4.75
production_0_wrenches = 15.0
production_1_pliers = 4.75
production_1_wrenches = 15.0
production_2_pliers = 8.5
production_2_wrenches = 12.5
production_3_pliers = 16.0
production_3_wrenches = 5.0
steelpurchase = 27.25
Total price =  863.25
